In [1]:
import os
import json

In [3]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise import SVD, evaluate
from surprise.model_selection import cross_validate, train_test_split

In [4]:
for line in open('user10reviews.json', 'r'):
    tenrevs = json.loads(line)

In [5]:
cut = [(r['user_id'], r['business_id'], r['stars']) for r in tenrevs]

In [14]:
print('Number of Reviews: '+str(len(cut)))

Number of Reviews: 2295089


In [18]:
ratings = [r['stars'] for r in tenrevs]

In [19]:
from collections import Counter
Counter(ratings)

Counter({3.0: 402943, 1.0: 172811, 2.0: 217710, 4.0: 749878, 5.0: 751747})

In [21]:
type(ratings[0])

float

In [7]:
businesses = [r['business_id'] for r in tenrevs]
unique_businesses = list(set(businesses))
users = [r['user_id'] for r in tenrevs]
unique_users = list(set(users))
print('Number of Unique Businesses: '+ str(len(unique_businesses)))
print('Number of Unique Users: '+ str(len(unique_users)))

Number of Unique Businesses: 73100
Number of Unique Users: 81416


In [8]:
indexed_users = {v: k for k, v in enumerate(unique_users)}
indexed_businesses = {v: k for k, v in enumerate(unique_businesses)}

In [9]:
indexed_data = list(map(lambda r: (indexed_users[r[0]], indexed_businesses[r[1]], r[2]), cut))

In [10]:
import csv
with open('finaldata.csv', 'w') as f:
    writer = csv.writer(f , lineterminator='\n')
    for tup in indexed_data:
        writer.writerow(tup)

In [11]:
file_path = os.path.expanduser('finaldata.csv')

In [12]:
reader = Reader(line_format='user item rating', sep=',')

In [13]:
data = Dataset.load_from_file(file_path, reader=reader)

In [11]:
data.split(n_folds=5)

In [16]:
svd = SVD()

In [17]:
evaluate(svd, data, measures=['RMSE', 'MAE'])

/anaconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/anaconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 1.0919
MAE:  0.8586
------------
Fold 2
RMSE: 1.0911
MAE:  0.8584
------------
Fold 3
RMSE: 1.0906
MAE:  0.8573
------------
Fold 4
RMSE: 1.0926
MAE:  0.8598
------------
Fold 5
RMSE: 1.0926
MAE:  0.8592
------------
------------
Mean RMSE: 1.0917
Mean MAE : 0.8587
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [1.0919205372150012,
                             1.0911238090890574,
                             1.0905641164574549,
                             1.0925744889239362,
                             1.0925621922072946],
                            'mae': [0.858605564047045,
                             0.8583596034838482,
                             0.8573090440636588,
                             0.8597598485326621,
                             0.8592491339068206]})

In [22]:
from surprise.model_selection import GridSearchCV

In [23]:
param_grid = {'n_factors': [50, 100, 200], 'biased': [True], 'lr_all': [0.005]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
# 'n_epochs': [10, 20, 50],'reg_all': [0.02, 0.05, 0.08]

In [24]:
gs.fit(data)

In [25]:
results = gs.cv_results

In [26]:
import pandas as pd
results_df = pd.DataFrame.from_dict(gs.cv_results)
results_df.to_csv('results.csv')

In [27]:
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_biased,param_lr_all
0,1.121815,1.123022,1.122295,1.122377,0.000496,2,0.894917,0.895330,0.895943,0.895397,0.000421,3,101.287115,12.756166,55.179468,56.157928,"{'biased': True, 'lr_all': 0.001}",True,0.001
1,1.095546,1.096588,1.096280,1.096138,0.000437,1,0.863294,0.863555,0.863951,0.863600,0.000270,1,122.237027,4.096766,125.242564,10.203921,"{'biased': True, 'lr_all': 0.005}",True,0.005
2,1.145895,1.146561,1.146847,1.146434,0.000399,3,0.887526,0.887307,0.888239,0.887690,0.000398,2,415.258017,388.697796,2648.197713,2215.837836,"{'biased': True, 'lr_all': 0.05}",True,0.050
3,2.665475,2.674566,2.661799,2.667280,0.005366,6,2.331765,2.341412,2.326670,2.333283,0.006113,6,129.907277,11.421524,178.133461,49.572544,"{'biased': False, 'lr_all': 0.001}",False,0.001
4,1.397292,1.400310,1.397937,1.398513,0.001297,5,1.110692,1.112792,1.110970,1.111485,0.000931,5,107.601211,8.414771,120.542719,10.293368,"{'biased': False, 'lr_all': 0.005}",False,0.005
5,1.246154,1.246630,1.246168,1.246317,0.000221,4,0.982923,0.982756,0.983399,0.983026,0.000272,4,105.472106,6.496184,81.059657,12.616613,"{'biased': False, 'lr_all': 0.05}",False,0.050


In [15]:
svdpp = SVDpp()

In [16]:
evaluate(svdpp, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVDpp.

------------
Fold 1
RMSE: 1.2451
MAE:  0.9911
------------
Fold 2
RMSE: 1.2439
MAE:  0.9904
------------
Fold 3
RMSE: 1.2435
MAE:  0.9902
------------
Fold 4
RMSE: 1.2439
MAE:  0.9898
------------
Fold 5
RMSE: 1.2423
MAE:  0.9894
------------
Fold 6
RMSE: 1.2454
MAE:  0.9917
------------
Fold 7
RMSE: 1.2423
MAE:  0.9888
------------
Fold 8
RMSE: 1.2440
MAE:  0.9906
------------
Fold 9
RMSE: 1.2444
MAE:  0.9909
------------
Fold 10
RMSE: 1.2435
MAE:  0.9898
------------
------------
Mean RMSE: 1.2438
Mean MAE : 0.9903
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [1.245125078509003,
                             1.243877565662575,
                             1.243534904258009,
                             1.2439133989460236,
                             1.2422777819164104,
                             1.2454152231851392,
                             1.242282298531058,
                             1.2440367384772635,
                             1.244350336377197,
                             1.243543967201196],
                            'mae': [0.9910805188094426,
                             0.9903994289118054,
                             0.9901717944360934,
                             0.989838450409646,
                             0.9894138213447615,
                             0.9917496974624626,
                             0.9887548745704519,
                             0.9905598968497563,
                             0.9908861770725602,
                           

In [18]:
nmf = NMF()

In [19]:
evaluate(nmf, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm NMF.

------------
Fold 1
RMSE: 1.3794
MAE:  1.0783
------------
Fold 2
RMSE: 1.3801
MAE:  1.0787
------------
Fold 3
RMSE: 1.3803
MAE:  1.0797
------------
Fold 4
RMSE: 1.3790
MAE:  1.0774
------------
Fold 5
RMSE: 1.3789
MAE:  1.0773
------------
Fold 6
RMSE: 1.3810
MAE:  1.0798
------------
Fold 7
RMSE: 1.3789
MAE:  1.0777
------------
Fold 8
RMSE: 1.3796
MAE:  1.0786
------------
Fold 9
RMSE: 1.3799
MAE:  1.0785
------------
Fold 10
RMSE: 1.3790
MAE:  1.0779
------------
------------
Mean RMSE: 1.3796
Mean MAE : 1.0784
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [1.3794350253411565,
                             1.3800631753244952,
                             1.3803470449633781,
                             1.3790474183963373,
                             1.3788653317907658,
                             1.3809921876160387,
                             1.3788922356505084,
                             1.3796261813374224,
                             1.3799325331811927,
                             1.3790269179343493],
                            'mae': [1.0782924691081697,
                             1.078739278586975,
                             1.07971219559644,
                             1.077435373740628,
                             1.0773078568785115,
                             1.0797835618332265,
                             1.0776545359866114,
                             1.0785939513347529,
                             1.0785215794765843,
                        